# Fuzzy ART com classes ordenadas

## Importando bibliotecas

In [1]:
%pip install artlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.datasets import USPS
from artlib import FuzzyART, FuzzyARTMAP, FusionART
from sklearn.metrics import classification_report, adjusted_rand_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import resample
from pyclustertend import vat
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Os limites são calculados de forma diferente quando se trata de imagens
def train_fuzzyART_images(X_train_subset, y_train_subset, X_test_subset, y_test_subset, n_dim):
    fuzzy_art_model = FuzzyART(rho=0.3, alpha=0.0, beta=1.0)

    lower_bounds = np.zeros(n_dim)
    upper_bounds = np.full(n_dim, 255.0)
    fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)

    train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train_subset)
    test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test_subset)

    fuzzy_art_model.partial_fit(train_X_fuzzy_art)
    fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

    return adjusted_rand_score(y_test_subset,fuzzy_art_predictions)


def train_fuzzyART(X_train_subset, y_train_subset, X_test_subset, y_test_subset):
    fuzzy_art_model = FuzzyART(rho=0.3, alpha=0.0, beta=1.0)

    X_combined = np.concatenate([X_train_subset, X_test_subset], axis=0)
    lower_bound, upper_bound = fuzzy_art_model.find_data_bounds(X_combined)
    fuzzy_art_model.set_data_bounds(lower_bound, upper_bound)

    train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train_subset)
    test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test_subset)

    fuzzy_art_model.partial_fit(train_X_fuzzy_art)
    fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

    return adjusted_rand_score(y_test_subset,fuzzy_art_predictions)


def generate_acc_matrix_fuzzyART(num_tasks, X_train_sorted, y_train_sorted, X_test_sorted, y_test_sorted, images):
    train_subsets = []
    test_subsets = []

    acc_matrix = [[0 for _ in range(num_tasks)] for _ in range(num_tasks)]

    for i in range(num_tasks):
        for j in range(num_tasks):
            # Classes até a i-ésima (inclusive)
            train_classes = torch.arange(0, i + 1)

            # Máscara de seleção para treino: todas as classes <= i
            mask_train = torch.isin(y_train_sorted, train_classes)
            X_train_subset = X_train_sorted[mask_train]
            y_train_subset = y_train_sorted[mask_train]

            # Máscara de seleção para teste: apenas a classe j
            mask_test = (y_test_sorted == j)
            X_test_subset = X_test_sorted[mask_test]
            y_test_subset = y_test_sorted[mask_test]

            # Armazena os subconjuntos (opcional)
            train_subsets.append((X_train_subset, y_train_subset))
            test_subsets.append((X_test_subset, y_test_subset))

            print(f"Classes de treino: {torch.unique(y_train_subset)}")
            print(f"Classes de teste: {torch.unique(y_test_subset)}")
            print(f"Tarefa {i}, Teste classe {j}, Tamanho do treino: {len(X_train_subset)}, Tamanho do teste: {len(X_test_subset)}")

            if len(X_train_subset) == 0 or len(X_test_subset) == 0:
                print("Problema: subconjunto de dados vazio")
                acc_matrix[i][j] = 0.0
                continue

            if(images):
                acc_matrix[i][j] = train_fuzzyART_images(
                                    X_train_subset,
                                    y_train_subset,  # não é usado na função
                                    X_test_subset,
                                    y_test_subset, 
                                    16 * 16)
            else:
                acc_matrix[i][j] = train_fuzzyART(
                                    X_train_subset,
                                    y_train_subset,  # não é usado na função
                                    X_test_subset,
                                    y_test_subset)
            
    return acc_matrix

## Baixando e ordenando dados

In [4]:
train_data = USPS(root='./USPS/', train=True, download=True)
test_data = USPS(root='./USPS/', train=False, download=True)

X_train = train_data.data
y_train = torch.tensor(train_data.targets, dtype=torch.long)

X_test = test_data.data
y_test = torch.tensor(test_data.targets, dtype=torch.long)

print(X_train.shape)

# as imagens do conjunto são 16x16  
n_dim = 16 * 16

#flatten images
X_train = X_train.reshape(-1, n_dim)
X_test = X_test.reshape(-1, n_dim)

# Ordenando dados
sorted_indices_train = torch.argsort(y_train)
X_train_sorted = X_train[sorted_indices_train]
y_train_sorted = y_train[sorted_indices_train]

sorted_indices_test = torch.argsort(y_test)
X_test_sorted = X_test[sorted_indices_test]
y_test_sorted = y_test[sorted_indices_test]

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [ ]:
print(y_train_sorted.unique())
print(y_test_sorted.unique())

y_train_sorted

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


tensor([0, 0, 0,  ..., 9, 9, 9])

In [ ]:
classes = y_train.unique()
print(classes)
num_tasks = len(classes)
print(num_tasks)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
10


## Fuzzy ART

Aprendizado não supervionado


In [ ]:
fuzzy_art_model = FuzzyART(rho=0.3, alpha=0.0, beta=1.0)

# Estabelecendo bounds
lower_bounds = np.zeros(n_dim) ## Isso foi feito com base no exemplo que está no github da biblioteca
upper_bounds = np.full(n_dim, 255.0)

fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)

# Normalização e complement coding
train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train)
test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test)

In [ ]:
fuzzy_art_model.partial_fit(train_X_fuzzy_art)
fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

In [ ]:
ARI = adjusted_rand_score(y_test,fuzzy_art_predictions)
print(f'O ARI do modelo FuzzyART foi: {ARI}')
num_clusters = fuzzy_art_model.n_clusters
print(f"O número de clusters foi: {num_clusters}")

O ARI do modelo FuzzyART foi: 0.04754222620721928
O número de clusters foi: 284


<font color="red">Por que tantos clusters?</font> 

In [ ]:
acc_matrix = generate_acc_matrix_fuzzyART(num_tasks, X_train_sorted, y_train_sorted, 
                                            X_test_sorted, y_test_sorted, 1)





Classes de treino: tensor([0, 1, 2, 3])
Classes de teste: tensor([8])
Tarefa 3, Teste classe 8, Tamanho do treino: 3588, Tamanho do teste: 166
Classes de treino: tensor([0, 1, 2, 3])
Classes de teste: tensor([9])
Tarefa 3, Teste classe 9, Tamanho do treino: 3588, Tamanho do teste: 177
Classes de treino: tensor([0, 1, 2, 3, 4])
Classes de teste: tensor([0])
Tarefa 4, Teste classe 0, Tamanho do treino: 4240, Tamanho do teste: 359
Classes de treino: tensor([0, 1, 2, 3, 4])
Classes de teste: tensor([1])
Tarefa 4, Teste classe 1, Tamanho do treino: 4240, Tamanho do teste: 264
Classes de treino: tensor([0, 1, 2, 3, 4])
Classes de teste: tensor([2])
Tarefa 4, Teste classe 2, Tamanho do treino: 4240, Tamanho do teste: 198
Classes de treino: tensor([0, 1, 2, 3, 4])
Classes de teste: tensor([3])
Tarefa 4, Teste classe 3, Tamanho do treino: 4240, Tamanho do teste: 166
Classes de treino: tensor([0, 1, 2, 3, 4])
Classes de teste: tensor([4])
Tarefa 4, Teste classe 4, Tamanho do treino: 4240, Tamanh

In [ ]:
np.set_printoptions(precision=6, suppress=False, floatmode='fixed')

print(np.array(acc_matrix))

[[0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]
 [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
  0.000000 0.000000]]


In [ ]:
print(f"Forward transfer: {forward_transfer(num_tasks, acc_matrix):.6f}")
print(f"Backward transfer: {backward_transfer(num_tasks, acc_matrix):.6f}")
print(f"Average Accuracy: {average_accuracy(num_tasks, acc_matrix):.6f}")


NameError: name 'forward_transfer' is not defined

<font color="red" oi>